## Building Hybrid Search in Qdrant

In [1]:
!pip install -q qdrant-client[fastembed]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.1 MB/s eta 0:00:00


In [9]:
!pip install -U sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [2]:
from qdrant_client import QdrantClient
from google.colab import userdata

client = QdrantClient(
    location = "https://b9d13b33-842f-4f65-8d2b-de07f146ebea.europe-west3-0.gcp.cloud.qdrant.io",
    api_key = userdata.get("qdrant_api_key")
)

In [6]:
from qdrant_client import models

collection_name = "hybrid_seach_demo"

client.create_collection(
    collection_name = collection_name,
    vectors_config = {
        "dense": models.VectorParams(
            distance = models.Distance.COSINE,
            size = 384
        ),
    },
    sparse_vectors_config = {
        "sparse": models.SparseVectorParams(
            modifier = models.Modifier.IDF
        )
    }
)

True

### Upload the data into the collection

In [7]:
documents = [
    "Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation.",
    "Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging.",
    "Brie cheese features a soft, creamy interior surrounded by an edible white rind.",
    "This French cheese has a flowing, buttery center encased in a bloomy white crust.",
    "Fresh mozzarella pairs beautifully with ripe tomatoes and basil leaves.",
    "Classic Margherita pizza topped with tomato sauce, mozzarella, and fresh basil.",
    "Parmesan requires at least 12 months of cave aging to develop its signature sharp taste.",
    "Parmigiano-Reggiano's distinctive piquant flavor comes from extended maturation in controlled environments.",
    "Grilled cheese sandwiches are the ultimate American comfort food for cold winter days.",
    "Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich.",
]

In [8]:
import uuid

In [11]:
client.upsert(
    collection_name = collection_name,
    points = [
        models.PointStruct(
            id = uuid.uuid4().hex,
            vector = {
                "dense": models.Document(
                    text = doc,
                    model = "sentence-transformers/all-MiniLM-L6-v2"
                ),
                "sparse": models.Document(
                    text = doc,
                    model = "Qdrant/bm25"
                )
            },
            payload = {"text": doc},
        )
        for i, doc in enumerate(documents)
    ]
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

spanish.txt: 0.00B [00:00, ?B/s]

russian.txt: 0.00B [00:00, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
def dense_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name = collection_name,
        query = models.Document(
            text = query,
            model = "sentence-transformers/all-MiniLM-L6-v2",
        ),
        using = "dense",
        limit = 3,
    )
    return response.points

In [13]:
def sparse_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name = collection_name,
        query = models.Document(
            text = query,
            model = "Qdrant/bm25",
        ),
        using = "sparse",
        limit = 3,
    )
    return response.points

### Run both methods on same queries and validate the output

In [14]:
queries = [
    "nutty aged cheese",
    "soft French cheese",
    "pizza ingredients",
    "a good lunch"
]

In [15]:
for query in queries:
    print("Query:", query)

    dense_results = dense_search(query)
    print("Dense Results:")
    for result in dense_results:
        print("\t-", result.payload["text"], result.score)

    sparse_results = sparse_search(query)
    print("Sparse Results:")
    for result in sparse_results:
        print("\t-", result.payload["text"], result.score)

    print()

Query: nutty aged cheese
Dense Results:
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 0.5829767
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.4764711
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 0.4505533
Sparse Results:
	- Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation. 5.1563325
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 3.0210652
	- Parmesan requires at least 12 months of cave aging to develop its signature sharp taste. 1.8819332

Query: soft French cheese
Dense Results:
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 0.6242111
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.60305476
	- Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich. 0.46846

### Hybrid Search with Reciprocal Rank Fusion

In [16]:
def rrf_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="sparse",
                limit=3,
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="sentence-transformers/all-MiniLM-L6-v2",
                ),
                using="dense",
                limit=3,
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=3,
    )

    return response.points

In [17]:
for query in queries:
    print("Query:", query)

    rrf_results = rrf_search(query)
    print("RRF Results:")
    for result in rrf_results:
        print("\t-", result.payload["text"], result.score)

Query: nutty aged cheese
RRF Results:
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 0.8333334
	- Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation. 0.5
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.33333334
Query: soft French cheese
RRF Results:
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 1.0
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.6666667
	- Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich. 0.5
Query: pizza ingredients
RRF Results:
	- Classic Margherita pizza topped with tomato sauce, mozzarella, and fresh basil. 1.0
	- Fresh mozzarella pairs beautifully with ripe tomatoes and basil leaves. 0.33333334
	- Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich. 0.25
Query: a good lunch
RRF Re